In [32]:
import torch
import torch.nn as nn
from ultralytics import YOLO

In [12]:
%run block.ipynb

In [38]:
reference = torch.load('yolov8x.pt')

In [39]:
reference

{'date': '2022-12-30T22:17:35.587856',
 'version': '8.0.0.dev0',
 'license': 'AGPL-3.0 License (https://ultralytics.com/license)',
 'docs': 'https://docs.ultralytics.com',
 'epoch': -1,
 'best_fitness': None,
 'model': DetectionModel(
   (model): Sequential(
     (0): Conv(
       (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU(inplace=True)
     )
     (1): Conv(
       (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU(inplace=True)
     )
     (2): C2f(
       (cv1): Conv(
         (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
         (act): SiLU(inplace=True)
      

In [28]:
class ClassifyV8(nn.Module):
    def __init__(self, in_channels, num_classes=10, device=None, dtype=None):
        super().__init__()
        self.hidden_channels = 1280
        self.conv = Conv(in_channels, out_channels=self.hidden_channels, kernel_size=(1, 1), stride=1, padding=0)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(self.hidden_channels, out_features=num_classes)

    def forward(self, x):
        if type(x) is list:
            x = torch.cat(x, 1)
        out = self.conv(x)
        out = self.pool(out)
        out = self.linear(out.flatten(1))
        return out if self.training else out.softmax(1)

variants taken from https://github.com/ultralytics/ultralytics/blob/07a5ff9ddca487581035b61ff7678c0f7e0f40d9/ultralytics/cfg/models/v8/yolov8.yaml

In [30]:
class Model(nn.Module):
    variants = {'n': {'d': 0.33, 'w': 0.25, 'mc': 1024},
                's': {'d': 0.33, 'w': 0.50, 'mc': 1024},
                'm': {'d': 0.67, 'w': 0.75, 'mc': 768},
                'l': {'d': 1.00, 'w': 1.00, 'mc': 512},
                'xl': {'d': 1.00, 'w': 1.25, 'mc': 512}}
    def __init__(self, device=None, dtype=None, 
                 residual_connection=False, CSP=False, add_hidden=False, bottleneck=1.0,
                 num_classes=1000):
        super().__init__()
        self.conv1 = Conv(3, out_channels=16, kernel_size=(3, 3), stride=(2, 2), 
                         padding=(1, 1), bias=False, 
                         device=device, dtype=dtype)
        self.conv2 = Conv(16, out_channels=32, kernel_size=(3, 3), stride=(2, 2), 
                          padding=(1, 1), bias=False,
                          device=device, dtype=dtype)
        self.c2f1 = C2f(32, out_channels=32, n=1, residual_connection=residual_connection, CSP=CSP, add_hidden=add_hidden, bottleneck=1.0)
        self.conv3 = Conv(32, out_channels=64, kernel_size=(3, 3), stride=(2, 2), 
                         padding=(1, 1), bias=False, 
                         device=device, dtype=dtype)
        self.c2f2 = C2f(64, out_channels=64, n=2, residual_connection=residual_connection, CSP=CSP, add_hidden=add_hidden, bottleneck=1.0)
        self.conv4 = Conv(64, out_channels=128, kernel_size=(3, 3), stride=(2, 2), 
                         padding=(1, 1), bias=False, 
                         device=device, dtype=dtype)
        self.c2f3 = C2f(128, out_channels=128, n=2, residual_connection=residual_connection, CSP=CSP, add_hidden=add_hidden, bottleneck=1.0)
        self.conv5 = Conv(128, out_channels=256, kernel_size=(3, 3), stride=(2, 2), 
                         padding=(1, 1), bias=False, 
                         device=device, dtype=dtype)
        self.c2f4 = C2f(256, out_channels=256, n=1, residual_connection=residual_connection, CSP=CSP, add_hidden=add_hidden, bottleneck=1.0)
        self.classify = Classify(256, num_classes=num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)

        out = self.c2f1(out)
        out = self.conv3(out)

        out = self.c2f2(out)
        out = self.conv4(out)

        out = self.c2f3(out)
        out = self.conv5(out)

        out = self.c2f4(out)

        out = self.classify(out)
        return out

In [16]:
inp = torch.rand(1, 3,640,640)

In [17]:
model1 = Model(num_classes=10)

In [18]:
out = model1(inp)

In [19]:
out.shape

torch.Size([1, 10])

In [20]:
total_params = sum(p.numel() for p in model1.parameters())
total_params

2850202

In [21]:
model2 = Model(num_classes=10, residual_connection=True, CSP=True)

In [22]:
out = model2(inp)

In [23]:
out.shape

torch.Size([1, 10])

In [24]:
total_params = sum(p.numel() for p in model2.parameters())
total_params

1397338

In [25]:
model3 = Model(num_classes=10, residual_connection=True, CSP=True, add_hidden=True)

In [26]:
out = model3(inp)

In [27]:
total_params = sum(p.numel() for p in model3.parameters())
total_params

1451098